# Rolling mean with CMIP6

## Import Python packages

In [ ]:
import s3fs
import xarray as xr
import intake
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

## Open CMIP6 online catalog

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

## Search for data 

In [ ]:
cat = col.search(
    source_id=["CESM2-WACCM"],
    experiment_id=["historical"],
    table_id=["AERmon"],
    variable_id=["so2"],
    member_id=["r1i1p1f1"],
)
cat.df

## Create a dictionary from the list of dataset

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={"use_cftime": True})

In [ ]:
list(dset_dict.keys())

## Open dataset

In [ ]:
dset = dset_dict["CMIP.NCAR.CESM2-WACCM.historical.AERmon.gn"]
dset

In [ ]:
dset.so2

## Compute Weighted average

In [ ]:
# Compute weights based on the xarray you pass
weights = np.cos(np.deg2rad(dset.lat))
weights.name = "weights"
# Compute weighted mean
var_weighted = dset.sel(lev=-1000, method="nearest").weighted(weights)
weighted_mean = var_weighted.mean(("lon", "lat"))

## Rolling mean
- Choose rolling time of 100

In [ ]:
%%time
weighted_mean.load()

In [ ]:
%%time
dpmean = (
    weighted_mean.chunk(chunks={"time": 100})
    .rolling({"time": 100}, min_periods=1, center=True)
    .mean()
)

## Visualize

In [ ]:
%%time
fig = plt.figure(1, figsize=[15, 7])

ax = plt.subplot(1, 1, 1)

weighted_mean.so2.plot(ax=ax, marker=".", linewidth=0, label="all points", alpha=0.1)

dpmean.so2.plot(ax=ax, marker="", linewidth=1, label="running mean [100 points]")
ax.legend()
ax.grid()
ax.set_ylabel(dset.so2.attrs["long_name"] + " (" + dset.so2.attrs["units"] + ")")
ax.set_xlabel("Time (year)")
ax.set_title("CMIP6 NCAR CESM2-WACCM historical (weighted rolling mean)")
plt.savefig("CMIP_NCAR_CESM2-WACCM_historical_weighted_rolling_mean.png")

## Save Results
To improve the reproducibility of our work (first for ourselves!), we need to keep track of all our work and ease its reuse. We will:
- Save this Jupyter notebook
- Save weighted mean
- Save rolling mean
- Save figure

We save intermediate results as netCDF files because they are small and ca be easily re-loaded (even if stored on cloud storage).

## Save results locally
- Useful for further analysis but can be lost if you close your JupyterLab or if there is any problem with your JupyterLab instance

In [ ]:
weighted_mean.to_netcdf("CMIP_NCAR_CESM2-WACCM_historical_weighted_mean.nc")

In [ ]:
dpmean.to_netcdf("CMIP_NCAR_CESM2-WACCM_historical_rolling_mean.nc")

## Save your results on NIRD (Norwegian infrastructure for Research Data)
- your credentials are in `$HOME/.aws/credentials` 
- check with your instructor to get the secret access key (replace XXX by the right key)

```
[default]
aws_access_key_id=forces2021-work
aws_secret_access_key=XXXXXXXXXXXX
aws_endpoint_url=https://forces2021.uiogeo-apps.sigma2.no/
```
<div class="alert alert-success alert-info">
    <b>It is important to save yoru results in a place that can last longer than a few days/weeks!</b>
</div>

In [ ]:
import s3fs

In [ ]:
fsg = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={"endpoint_url": "https://forces2021.uiogeo-apps.sigma2.no/"},
)

## Set "remote" path (update annefou by your username) and save weighted mean as netCDF file

In [ ]:
s3_path = "s3://work/annefou/CMIP_NCAR_CESM2-WACCM_historical_weighted_mean.nc"
print(s3_path)

In [ ]:
with fsg.open(s3_path, "wb") as f:
    f.write(weighted_mean.to_netcdf(None))

## Save rolling mean to remote location (update annefou with your username)

In [ ]:
s3_path = "s3://work/annefou/CMIP_NCAR_CESM2-WACCM_historical_rolling_mean.nc"
print(s3_path)

In [ ]:
with fsg.open(s3_path, "wb") as f:
    f.write(dpmean.to_netcdf(None))

## Upload existing png file to remote s3 location

In [ ]:
s3_path = "s3://work/annefou/CMIP_NCAR_CESM2-WACCM_historical_weighted_rolling_mean.png"
print(s3_path)

In [ ]:
fsg.put("CMIP_NCAR_CESM2-WACCM_historical_weighted_rolling_mean.png", s3_path)